In [62]:
import pandas as pd
import requests, json, time
from bs4 import BeautifulSoup

In [ ]:
def get_s1_filings():
    base_url = "https://efts.sec.gov/LATEST/search-index"

    start_date = "2024-01-02"
    end_date = "2024-12-31"

    requests_per_second = 10
    delay = 1.0 / requests_per_second
    
    start_from = 0
    size = 100
    all_filings = []

    headers = {
        "User-Agent": "MyApp/1.0 (ruoyu@lockrmail.com)",
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    
    initial_url = f"{base_url}?dateRange=custom&category=custom&startdt={start_date}&enddt={end_date}&forms=S-1&page=1&from=0"
    response = requests.get(initial_url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        total_hits = data.get("hits", {}).get("total", {}).get("value", 0)
        print(f"Total S-1 filings found: {total_hits}")
        
        while start_from < total_hits:
            url = f"{base_url}?dateRange=custom&category=custom&startdt={start_date}&enddt={end_date}&forms=S-1&page={start_from // size + 1}&from={start_from}"
            response = requests.get(url, headers=headers)
            time.sleep(delay)
            
            if response.status_code == 200:
                data = response.json()
                filings = data.get("hits", {}).get("hits", [])
                
                for filing in filings:
                    source_data = filing.get("_source")
                    all_filings.append({
                        "display_names": source_data.get("display_names"),
                        "file_date": source_data.get("file_date"),
                        "file_type": source_data.get("file_type"),
                        "biz_locations": source_data.get("biz_locations"),
                        "sequence": source_data.get("sequence"),
                        "inc_states": source_data.get("inc_states"),
                    })
                
                start_from += size 
            else:
                print(f"Error: Received status code {response.status_code}")
                print(response.text)
                break
    else:
        print(f"Error: Received status code {response.status_code}")
        print(response.text)
    
    return all_filings

In [64]:
list_of_filings = get_s1_filings()

Total S-1 filings found: 2663


2663

In [3]:
url = "https://efts.sec.gov/LATEST/search-index"
headers = {
    "User-Agent": "MyApp/1.0 (ruoyu@lockrmail.com)",
    "Accept": "application/json"
}
params = {
        "q": "FORM-TYPE:S-1 AND filed:[2024-01-01 TO 2024-12-31]",  # Search for S-1 filings in 2024
        "from": 0,
        "size": 100  # Max results (increase if needed)
    }

In [55]:
url = "https://efts.sec.gov/LATEST/search-index?dateRange=custom&category=custom&startdt=2024-01-02&enddt=2024-12-31&forms=S-1&page=1&from=0"

In [56]:
response = requests.get(url, headers=headers)

In [73]:
response.json().get("hits").get("hits")[0]

{'_index': 'edgar_file',
 '_id': '0001640334-24-001974:aen_s1.htm',
 '_score': None,
 '_source': {'ciks': ['0002033750'],
  'period_ending': None,
  'file_num': ['333-284102'],
  'display_names': ['AEN Group Ltd.  (CIK 0002033750)'],
  'xsl': None,
  'sequence': 1,
  'root_forms': ['S-1'],
  'file_date': '2024-12-31',
  'biz_states': ['A0'],
  'sics': [],
  'form': 'S-1',
  'adsh': '0001640334-24-001974',
  'film_num': ['241593326'],
  'biz_locations': ['Calgary, A0'],
  'file_type': 'S-1',
  'file_description': 'FORM S-1',
  'inc_states': [],
  'items': []},
 'sort': [1735603200000]}

In [77]:
response.json().get("hits").get("hits")[0].get("_source")

{'ciks': ['0002033750'],
 'period_ending': None,
 'file_num': ['333-284102'],
 'display_names': ['AEN Group Ltd.  (CIK 0002033750)'],
 'xsl': None,
 'sequence': 1,
 'root_forms': ['S-1'],
 'file_date': '2024-12-31',
 'biz_states': ['A0'],
 'sics': [],
 'form': 'S-1',
 'adsh': '0001640334-24-001974',
 'film_num': ['241593326'],
 'biz_locations': ['Calgary, A0'],
 'file_type': 'S-1',
 'file_description': 'FORM S-1',
 'inc_states': [],
 'items': []}